# <a id='toc1_'></a>[Анализ удовлетворённости клиентов авиакомпании с помощью PySpark](#toc0_)

Цель проекта: определить влияние различных факторов на уровень удавлетворённости клиентов.

[Ссылка на датасет](https://www.kaggle.com/datasets/sjleshrac/airlines-customer-satisfaction/data)

## Загрузка данных

In [1]:
# импорт библиотек
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark import  SparkContext
from pyspark.sql.functions import col, lit
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.stat import Statistics
from pyspark.mllib.linalg import Vectors

In [2]:
spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_air')\
        .getOrCreate()

In [3]:
# чтение CSV файла
data = spark.read.csv(
    'invistico_airline.csv',
    sep=',',
    header=True,
    inferSchema=True
)

In [4]:
data.printSchema() # схема данных

root
 |-- satisfaction: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Customer Type: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Type of Travel: string (nullable = true)
 |-- Class: string (nullable = true)
 |-- Flight Distance: integer (nullable = true)
 |-- Seat comfort: integer (nullable = true)
 |-- Departure/Arrival time convenient: integer (nullable = true)
 |-- Food and drink: integer (nullable = true)
 |-- Gate location: integer (nullable = true)
 |-- Inflight wifi service: integer (nullable = true)
 |-- Inflight entertainment: integer (nullable = true)
 |-- Online support: integer (nullable = true)
 |-- Ease of Online booking: integer (nullable = true)
 |-- On-board service: integer (nullable = true)
 |-- Leg room service: integer (nullable = true)
 |-- Baggage handling: integer (nullable = true)
 |-- Checkin service: integer (nullable = true)
 |-- Cleanliness: integer (nullable = true)
 |-- Online boarding: integer (nullable = true)

In [5]:
data.count() # число строк

129880

In [6]:
# обзор пропусков
Dict_Null = {col:data.filter(data[col].isNull()).count() for col in data.columns}
Dict_Null

{'satisfaction': 0,
 'Gender': 0,
 'Customer Type': 0,
 'Age': 0,
 'Type of Travel': 0,
 'Class': 0,
 'Flight Distance': 0,
 'Seat comfort': 0,
 'Departure/Arrival time convenient': 0,
 'Food and drink': 0,
 'Gate location': 0,
 'Inflight wifi service': 0,
 'Inflight entertainment': 0,
 'Online support': 0,
 'Ease of Online booking': 0,
 'On-board service': 0,
 'Leg room service': 0,
 'Baggage handling': 0,
 'Checkin service': 0,
 'Cleanliness': 0,
 'Online boarding': 0,
 'Departure Delay in Minutes': 0,
 'Arrival Delay in Minutes': 393}

## Проверка данных и предобработка

In [7]:
# корректировка названий колонок
data = data.select([F.col(x).alias(x.lower()) for x in data.columns])
data = data.select([F.col(x).alias(x.replace(' ', '_')) for x in data.columns])
data = data.select([F.col(x).alias(x.replace('/', '_')) for x in data.columns])
data = data.select([F.col(x).alias(x.replace('-', '_')) for x in data.columns])
data.show(2)

+------------+------+--------------+---+---------------+--------+---------------+------------+---------------------------------+--------------+-------------+---------------------+----------------------+--------------+----------------------+----------------+----------------+----------------+---------------+-----------+---------------+--------------------------+------------------------+
|satisfaction|gender| customer_type|age| type_of_travel|   class|flight_distance|seat_comfort|departure_arrival_time_convenient|food_and_drink|gate_location|inflight_wifi_service|inflight_entertainment|online_support|ease_of_online_booking|on_board_service|leg_room_service|baggage_handling|checkin_service|cleanliness|online_boarding|departure_delay_in_minutes|arrival_delay_in_minutes|
+------------+------+--------------+---+---------------+--------+---------------+------------+---------------------------------+--------------+-------------+---------------------+----------------------+--------------+-------

## Работа с пропусками

In [8]:
data = data.na.fill(0) # замена пропусков нулями

In [9]:
# проверка пропусков
Dict_Null = {col:data.filter(data[col].isNull()).count() for col in data.columns}
Dict_Null

{'satisfaction': 0,
 'gender': 0,
 'customer_type': 0,
 'age': 0,
 'type_of_travel': 0,
 'class': 0,
 'flight_distance': 0,
 'seat_comfort': 0,
 'departure_arrival_time_convenient': 0,
 'food_and_drink': 0,
 'gate_location': 0,
 'inflight_wifi_service': 0,
 'inflight_entertainment': 0,
 'online_support': 0,
 'ease_of_online_booking': 0,
 'on_board_service': 0,
 'leg_room_service': 0,
 'baggage_handling': 0,
 'checkin_service': 0,
 'cleanliness': 0,
 'online_boarding': 0,
 'departure_delay_in_minutes': 0,
 'arrival_delay_in_minutes': 0}

## Проверка данных

In [10]:
data.head(3) # шапка таблицы

[Row(satisfaction='satisfied', gender='Female', customer_type='Loyal Customer', age=65, type_of_travel='Personal Travel', class='Eco', flight_distance=265, seat_comfort=0, departure_arrival_time_convenient=0, food_and_drink=0, gate_location=2, inflight_wifi_service=2, inflight_entertainment=4, online_support=2, ease_of_online_booking=3, on_board_service=3, leg_room_service=0, baggage_handling=3, checkin_service=5, cleanliness=3, online_boarding=2, departure_delay_in_minutes=0, arrival_delay_in_minutes=0),
 Row(satisfaction='satisfied', gender='Male', customer_type='Loyal Customer', age=47, type_of_travel='Personal Travel', class='Business', flight_distance=2464, seat_comfort=0, departure_arrival_time_convenient=0, food_and_drink=0, gate_location=3, inflight_wifi_service=0, inflight_entertainment=2, online_support=2, ease_of_online_booking=3, on_board_service=4, leg_room_service=4, baggage_handling=4, checkin_service=2, cleanliness=3, online_boarding=2, departure_delay_in_minutes=310, a

In [11]:
data.describe().show() # описательная статистика

+-------+------------+------+-----------------+------------------+---------------+--------+-----------------+-----------------+---------------------------------+------------------+------------------+---------------------+----------------------+------------------+----------------------+------------------+------------------+-----------------+------------------+------------------+------------------+--------------------------+------------------------+
|summary|satisfaction|gender|    customer_type|               age| type_of_travel|   class|  flight_distance|     seat_comfort|departure_arrival_time_convenient|    food_and_drink|     gate_location|inflight_wifi_service|inflight_entertainment|    online_support|ease_of_online_booking|  on_board_service|  leg_room_service| baggage_handling|   checkin_service|       cleanliness|   online_boarding|departure_delay_in_minutes|arrival_delay_in_minutes|
+-------+------------+------+-----------------+------------------+---------------+--------+-----

In [12]:
data.toPandas().nunique() # число уникальных значений

satisfaction                            2
gender                                  2
customer_type                           2
age                                    75
type_of_travel                          2
class                                   3
flight_distance                      5398
seat_comfort                            6
departure_arrival_time_convenient       6
food_and_drink                          6
gate_location                           6
inflight_wifi_service                   6
inflight_entertainment                  6
online_support                          6
ease_of_online_booking                  6
on_board_service                        6
leg_room_service                        6
baggage_handling                        5
checkin_service                         6
cleanliness                             6
online_boarding                         6
departure_delay_in_minutes            466
arrival_delay_in_minutes              472
dtype: int64

In [13]:
data.toPandas()['satisfaction'].value_counts() # распределение основного показателя

satisfaction
satisfied       71087
dissatisfied    58793
Name: count, dtype: int64

Удовлетворённые клиенты преобладают

In [14]:
# расчёт совокупной метрики
data = data.withColumn('total', data.seat_comfort + data.departure_arrival_time_convenient + \
               data.food_and_drink + data.gate_location + data.inflight_wifi_service + \
               data.inflight_entertainment + data.online_support + data.ease_of_online_booking + \
               data.on_board_service + data.leg_room_service + data.baggage_handling + \
               data.checkin_service + data.cleanliness + data.online_boarding)
data.show(5)

+------------+------+--------------+---+---------------+--------+---------------+------------+---------------------------------+--------------+-------------+---------------------+----------------------+--------------+----------------------+----------------+----------------+----------------+---------------+-----------+---------------+--------------------------+------------------------+-----+
|satisfaction|gender| customer_type|age| type_of_travel|   class|flight_distance|seat_comfort|departure_arrival_time_convenient|food_and_drink|gate_location|inflight_wifi_service|inflight_entertainment|online_support|ease_of_online_booking|on_board_service|leg_room_service|baggage_handling|checkin_service|cleanliness|online_boarding|departure_delay_in_minutes|arrival_delay_in_minutes|total|
+------------+------+--------------+---+---------------+--------+---------------+------------+---------------------------------+--------------+-------------+---------------------+----------------------+----------

In [15]:
data.dtypes # типы данных столбцов

[('satisfaction', 'string'),
 ('gender', 'string'),
 ('customer_type', 'string'),
 ('age', 'int'),
 ('type_of_travel', 'string'),
 ('class', 'string'),
 ('flight_distance', 'int'),
 ('seat_comfort', 'int'),
 ('departure_arrival_time_convenient', 'int'),
 ('food_and_drink', 'int'),
 ('gate_location', 'int'),
 ('inflight_wifi_service', 'int'),
 ('inflight_entertainment', 'int'),
 ('online_support', 'int'),
 ('ease_of_online_booking', 'int'),
 ('on_board_service', 'int'),
 ('leg_room_service', 'int'),
 ('baggage_handling', 'int'),
 ('checkin_service', 'int'),
 ('cleanliness', 'int'),
 ('online_boarding', 'int'),
 ('departure_delay_in_minutes', 'int'),
 ('arrival_delay_in_minutes', 'int'),
 ('total', 'int')]

## Обзор совокупного показателя

In [16]:
data.agg({"total": "max"}).collect()[0][0] # максимальное значение основной метрики

70

In [17]:
data.filter(col('total') == lit(70)).show(5) # записи с максимальным значением основной метрики

+------------+------+--------------+---+---------------+--------+---------------+------------+---------------------------------+--------------+-------------+---------------------+----------------------+--------------+----------------------+----------------+----------------+----------------+---------------+-----------+---------------+--------------------------+------------------------+-----+
|satisfaction|gender| customer_type|age| type_of_travel|   class|flight_distance|seat_comfort|departure_arrival_time_convenient|food_and_drink|gate_location|inflight_wifi_service|inflight_entertainment|online_support|ease_of_online_booking|on_board_service|leg_room_service|baggage_handling|checkin_service|cleanliness|online_boarding|departure_delay_in_minutes|arrival_delay_in_minutes|total|
+------------+------+--------------+---+---------------+--------+---------------+------------+---------------------------------+--------------+-------------+---------------------+----------------------+----------

Все клиенты, поставившие максимальный бал, были удовлетворены

In [18]:
data.select(
    F.min(F.col("total")).alias("MIN")).limit(1).collect()[0].MIN # минимальное значение основной метрики

15

In [19]:
data.filter(col('total') < lit(18)).show(50) # записи с низким значением основной метрики

+------------+------+-----------------+---+---------------+--------+---------------+------------+---------------------------------+--------------+-------------+---------------------+----------------------+--------------+----------------------+----------------+----------------+----------------+---------------+-----------+---------------+--------------------------+------------------------+-----+
|satisfaction|gender|    customer_type|age| type_of_travel|   class|flight_distance|seat_comfort|departure_arrival_time_convenient|food_and_drink|gate_location|inflight_wifi_service|inflight_entertainment|online_support|ease_of_online_booking|on_board_service|leg_room_service|baggage_handling|checkin_service|cleanliness|online_boarding|departure_delay_in_minutes|arrival_delay_in_minutes|total|
+------------+------+-----------------+---+---------------+--------+---------------+------------+---------------------------------+--------------+-------------+---------------------+----------------------+-

Не все клиенты, поставившие низкий бал, были неудовлетворены

## Средние показатели

In [20]:
all = data.groupBy("satisfaction", "gender", "customer_type", "type_of_travel", "class") \
    .agg(F.round(F.avg("total"), 2).alias("total"),
         F.round(F.avg("seat_comfort"), 2).alias("seat_comfort"), 
         F.round(F.avg("departure_arrival_time_convenient"), 2).alias("departure_arrival_time_convenient"), 
         F.round(F.avg("food_and_drink"), 2).alias("food_and_drink"),
         F.round(F.avg("gate_location"), 2).alias("gate_location"), 
         F.round(F.avg("inflight_wifi_service"), 2).alias("inflight_wifi_service"), 
         F.round(F.avg("inflight_entertainment"), 2).alias("inflight_entertainment"), 
         F.round(F.avg("online_support"), 2).alias("online_support"), 
         F.round(F.avg("ease_of_online_booking"), 2).alias("ease_of_online_booking"), 
         F.round(F.avg("on_board_service"), 2).alias("on_board_service"), 
         F.round(F.avg("leg_room_service"), 2).alias("leg_room_service"), 
         F.round(F.avg("baggage_handling"), 2).alias("baggage_handling"), 
         F.round(F.avg("checkin_service"), 2).alias("checkin_service"), 
         F.round(F.avg("cleanliness"), 2).alias("cleanliness"), 
         F.round(F.avg("online_boarding"), 2).alias("online_boarding")
      ).orderBy(
          ["satisfaction", "customer_type", "gender", "type_of_travel", "class"], 
          ascending=False).show(43, truncate=True)

+------------+------+-----------------+---------------+--------+-----+------------+---------------------------------+--------------+-------------+---------------------+----------------------+--------------+----------------------+----------------+----------------+----------------+---------------+-----------+---------------+
|satisfaction|gender|    customer_type| type_of_travel|   class|total|seat_comfort|departure_arrival_time_convenient|food_and_drink|gate_location|inflight_wifi_service|inflight_entertainment|online_support|ease_of_online_booking|on_board_service|leg_room_service|baggage_handling|checkin_service|cleanliness|online_boarding|
+------------+------+-----------------+---------------+--------+-----+------------+---------------------------------+--------------+-------------+---------------------+----------------------+--------------+----------------------+----------------+----------------+----------------+---------------+-----------+---------------+
|   satisfied|  Male|disl

Самый низкий совокупный показатель в среднем у удовлетворённых клиентов среди женщин и мужчин, относящихся к нелояльным, имеющих целью бизнес-путешествие и с классом Эконом Плюс, а самый высокий - среди лояльных с аналогичной целью и классом Бизнес.

Самый высокий совокупный показатель в среднем у неудовлетворённых клиентов среди женщин, относящихся к лояльным, имеющих целью индивидуальное путешествие, а самый низкий - среди лояльных мужчин с целью бизнес-путешествия.

По отдельным критериям нет среднего значения показателя ниже 2 (самый низкий зафиксирован по оценке еды (2.1) и напитков среди неудовлетворённых нелояльных женщин в индивидуальном путешествии класса Эконом), а самый высокий показатель фиксируется по комфорту сидения (4.4 у удовлетворённых нелояльных женщин при индивидуальном путешествии эконом класса).

## Матрица корреляций

In [21]:
data_new = data[['flight_distance', 'seat_comfort', 'departure_arrival_time_convenient', \
               'food_and_drink', 'gate_location', 'inflight_wifi_service', \
               'inflight_entertainment', 'online_support', 'ease_of_online_booking', \
               'on_board_service', 'leg_room_service', 'baggage_handling', \
               'checkin_service', 'cleanliness', 'online_boarding', \
               'departure_delay_in_minutes', 'arrival_delay_in_minutes', 'total']]

vector_col = 'satisfaction'
assembler = VectorAssembler(inputCols=data_new.columns, outputCol=vector_col)
df_vector = assembler.transform(data_new).select(vector_col)

corr_matrix = Correlation.corr(df_vector, vector_col)

co = corr_matrix.collect()[0]['pearson({})'.format(vector_col)].values
co

array([ 1.00000000e+00, -4.24087595e-02,  1.35028774e-03, -4.64920013e-03,
       -2.40888816e-03,  1.22302223e-02, -2.75423117e-02, -3.20224837e-02,
       -2.22992839e-02, -3.25590394e-02, -3.03157699e-02,  1.97683971e-02,
        2.45565044e-03,  9.43181254e-03,  9.60352495e-03,  1.12555440e-01,
        1.09733489e-01, -2.01768143e-02, -4.24087595e-02,  1.00000000e+00,
        4.34951201e-01,  7.15997131e-01,  4.05299613e-01,  1.29130660e-01,
        4.26447115e-01,  1.20278266e-01,  2.11530683e-01,  1.21148837e-01,
        1.36135696e-01,  1.18488629e-01,  4.12136522e-02,  1.08475074e-01,
        1.30396484e-01, -2.38171016e-02, -2.54626380e-02,  5.95781118e-01,
        1.35028774e-03,  4.34951201e-01,  1.00000000e+00,  5.27381194e-01,
        5.44258524e-01, -1.97370209e-03,  7.72029136e-02, -5.46247124e-04,
        1.75548932e-03,  6.04463693e-02,  2.94218568e-02,  6.73961551e-02,
        6.27937697e-02,  6.71257030e-02, -6.23459767e-04,  4.39513128e-03,
        2.49975211e-03,  

In [22]:
myGraph_vector = assembler.transform(data_new).select(vector_col)
matrix = Correlation.corr(myGraph_vector, vector_col).collect()[0][0]
corrmatrix = matrix.toArray().tolist()
print(corrmatrix)

[[1.0, -0.04240875945372055, 0.0013502877430506782, -0.004649200130602286, -0.0024088881570628333, 0.012230222348860852, -0.027542311731235426, -0.03202248370455009, -0.02229928385112214, -0.032559039424103604, -0.030315769893668423, 0.01976839711421719, 0.0024556504445183125, 0.009431812539732215, 0.009603524945972946, 0.11255544011216746, 0.10973348892058468, -0.020176814289956994], [-0.04240875945372055, 1.0, 0.43495120128020737, 0.7159971305520978, 0.40529961252724567, 0.1291306603384352, 0.42644711501534877, 0.12027826578174698, 0.21153068290420493, 0.1211488374279974, 0.1361356961505449, 0.11848862920676026, 0.041213652155744955, 0.10847507365056178, 0.13039648409007182, -0.0238171015881213, -0.025462637957651024, 0.5957811184347558], [0.0013502877430506782, 0.43495120128020737, 1.0, 0.5273811938548395, 0.5442585238078219, -0.0019737020867547908, 0.07720291358769626, -0.0005462471241289544, 0.0017554893171509804, 0.06044636928805721, 0.029421856812452133, 0.06739615507603047, 0.0